In [1]:
import time
import csv
import json
import random
import time
import Levenshtein
import wikitextparser as wtp
from wiki_dump_reader import iterate
from tqdm import tqdm

from extraction_croissement import *

In [2]:
start_loading_data_time = time.time()

# Loading des données sauvegardées dans la mémoire ram
g_book_ADP = rdflib.Graph()
g_author_ADP = rdflib.Graph()
ADP_book_graph = g_book_ADP.parse("../Graphes/grapheADPLivres.rdf")
ADP_author_graph = g_author_ADP.parse("../Graphes/grapheADPAuteurs.rdf")
ADP_books = get_ADP_books(g_book_ADP, g_author_ADP)
ADP_loading_time = time.time()
print("ADP_loading_time: ", ADP_loading_time - start_loading_data_time)

g_item_DL = rdflib.Graph()
book_graph_DL = g_item_DL.parse("../Graphes/grapheDepotLegal.rdf")
DL_books = get_depot_legal_book(g_item_DL)
DL_loading_time = time.time()
print("DL_loading_time: ", DL_loading_time - ADP_loading_time)

g_item_ILE = rdflib.Graph()
item_graph_ILE = g_item_ILE.parse("../Graphes/grapheILE.rdf")
ILE_books = get_ILE_book(g_item_ILE)
ILE_loading_time = time.time()
print("ILE_loading time: ", ILE_loading_time - DL_loading_time)

books_Hurtubise_file = open("./Hurtubise/Exportation-Hurtubise.csv", "r", encoding='ISO-8859-1')
csv_reader = csv.DictReader(books_Hurtubise_file, delimiter=',', fieldnames=[
    "Editeur", "ISBN Papier", "ISBN PDF", "ISBN epub", "Titre", "Sous - titre", "Titre de la serie",
    "Contributeurs", "Contributeur(premier)", "Langue", "Langue Origine", "Resume", "Nombre de pages",
    "Date de parution", "Annee de parution", "Sujet  THEMA principal", "Sujet THEMA",
    "Quantificateur Georaphique", "Quantificateur de langue", "Quantificateur Historique", "Niveau soclaire FR",
    "Niveau scolaire QC", "Cycle scolaire FR", "Niveau de lecture", "Echele CECR", "Quantificateur d'interet",
    "Quantificateur d'age", "Quantificateur de style", "Classification Editoriale", "Mots cles"

])
Hurtubise_books = get_Hurtubise_books(csv_reader)
books_Hurtubise_file.close()

authors_ILE_file = open("./ILE/auteurs_ILE_comma_separated.csv", 'r', encoding='ISO-8859-1')
csv_reader = csv.DictReader(authors_ILE_file, delimiter=',', fieldnames=[
    'uri', 'nom', 'bio', 'genres', 'site', 'pseudonyme'])
authors_ILE = [x for x in csv_reader]
authors_ILE_file.close()

authors_wikidata_file = open("./Wikidata/ecrivains_wikidata_comma_separated.csv", 'r', encoding='ISO-8859-1')
csv_reader = csv.DictReader(authors_wikidata_file, delimiter=',', fieldnames=[
    'nom', 'uri'])
authors_wikidata = [x for x in csv_reader]
authors_wikidata_file.close()

authors_DBpedia_file = open("./DBpedia/ecrivains_dbpedia_fr.txt", "r", encoding='ISO-8859-1')
csv_reader = csv.DictReader(authors_DBpedia_file, delimiter=';', fieldnames=[
    'uri', 'nom'])
authors_DBpedia = [x for x in csv_reader]
authors_DBpedia_file.close()

babelioJsonBooks = open("./Babelio/babelio_livres.json", "r")
Babelio_books = get_Babelio_books(json.load(babelioJsonBooks))
babelioJsonBooks.close()

babelioJsonAuthor = open("./Babelio/babelio_auteurs.json", "r")
Babelio_authors = get_Babelio_books(json.load(babelioJsonAuthor))
babelioJsonAuthor.close()

loading_data_time = time.time()
print("loading_data_time: ", loading_data_time - start_loading_data_time)

all_books = ADP_books + ILE_books + Hurtubise_books + Babelio_books + DL_books
random.shuffle(all_books)

ADP_loading_time:  12.159992456436157
DL_loading_time:  48.21831011772156
ILE_loading time:  14.360928058624268
loading_data_time:  76.2224953174591


In [3]:
author_ls = {}
print('recoupement des auteurs')
for book in tqdm(all_books, total=len(all_books)):
    if len(book['author_raw']) == 1 and isinstance(book['author_raw'][0], list):
        book['author_raw'] = book['author_raw'][0]
    for author in book['author_raw']:
        if author.replace('"', '') not in author_ls:
            author_ls[author.replace('"', '')] = [book]
        else:
            author_ls[author.replace('"', '')].append(book)

100%|██████████| 102985/102985 [00:00<00:00, 333022.17it/s]


recoupement des auteurs


In [6]:
# def parse_Info(str):
#     result = re.search(r'\{\{Infobox (Ouvrage|Livre)(.|\n)+\}\}', str)
#     info_box_str = result.group()
#     nb_bracket = 0
#     for index, caractere in enumerate(info_box_str):
#         if caractere == '{':
#             nb_bracket += 1
#         elif caractere == '}':
#             nb_bracket -= 1
#         if nb_bracket == 0:
#             return info_box_str[:index]
#
# def get_info(text):
#     raw_infos = parse_Info(text)
#     infos = {}
#     for info in raw_infos.split('\n')[1:-1]:
#         test = info.split('=')
#         key = info.split('=')[0]
#         value = info.split('=')[1]
#         infos[key] = value
#     return infos

# couple_books = []
# with open('wikipedia/fr_dumps_wikipedia_books.json') as json_file:
#     data = json.load(json_file)
#     for title, text in tqdm(data.items(), total=len(data.items())):
#         for book in all_books:
#             dist_titre = Levenshtein.distance(book['title'], normalize(title))
#             if dist_titre < max(1, min(len(book['title']), len(title)) / 4):
#                 print("titre DB: ", book['title'], "titre de wiki:", title)
#                 new_couple = {
#                         'titre DB': book['title'],
#                         'titre wiki': title,
#                         'book DB': book,
#                     }
#                 try:
#                     new_couple['book wiki'] = get_info(text)
#                 except:
#                     new_couple['book wiki'] = text
#                 couple_books.append(new_couple)
#
# with open('couple_wiki_books.json', 'w') as outfile:
#     json.dump(couple_books, outfile)

# confirmed_couple = []
# unconfirmed_couple = []
# for couple in tqdm(couple_books, total=len(couple_books)):
#     if isinstance(couple['book wiki'], dict):
#         keys = list(couple['book wiki'].keys())
#         for key in keys:
#             couple['book wiki'][normalize(key)] = normalize(couple['book wiki'][key])
#         if 'auteur' in couple['book wiki'] and couple['book wiki']['auteur']:
#             correspondance = False
#             for author in couple['book DB']['author']:
#                 test = (couple['book wiki']['auteur'], author)
#                 dist_auteur = Levenshtein.distance(couple['book wiki']['auteur'], author)
#                 if dist_auteur < max(1, min(len(couple['book wiki']['auteur']), len(author)) / 4):
#                     confirmed_couple.append(couple)
#                     correspondance = True
#                     break
#             if not correspondance:
#                 unconfirmed_couple.append(couple)
#     else:
#         # print('a faire a la main: titre DB=', couple['titre DB'],' titre wiki=', couple['titre wiki'], ' wiki text=', couple['book wiki'][:200])
#         pass
# print('nombre de correspondance: ', len(confirmed_couple))
# print('nombre de distinct: ', len(unconfirmed_couple))
# print('confirmed_couple:')
# print(json.dumps(confirmed_couple, indent=2))
# print('unconfirmed_couple:')
# print(json.dumps(unconfirmed_couple, indent=2))

In [11]:
famous = ['Michel Tremblay', 'Anne Hébert', 'Gabrielle Roy', 'Marie Cardinal', 'Réjean Ducharme',
                'Jacques Ferron', 'Victor-Lévy Beaulieu', 'Marcel Dubé', 'Yves Thériault', 'Jacques Poulin',
                'André Langevin']

def split_info_box(str):
    nb_bracket = 0
    start_index = str.find("{{Infobox")
    for index, caractere in enumerate(str[start_index:]):
        if caractere == '{':
            test_str = str[start_index:start_index + index + 1]
            nb_bracket += 1
        elif caractere == '}':
            test_str = str[start_index:start_index + index + 1]
            nb_bracket -= 1
        if nb_bracket == 0:
            return str[:start_index + index + 1], str[start_index + index + 2:]

def get_info(raw_infos):
    if not raw_infos:
        return text
    infos = {}
    for info in re.split(r"\n\||\n\}", raw_infos, flags=re.DOTALL)[1:-1]:
        if len(info.split('=')) > 1:
            key, value = info.split('=')[0], info.split('=')[1]
            infos[key] = value
    return infos

def get_list_from_section(section):
    if section.get_lists():
        return [re.sub(r"\'\'", "", item) for list in section.get_lists() for item in list.items]
    else:
        return None


def parse_section(section, title_ls):

    section_dict = {'content': section.contents,
                    'list': get_list_from_section(section),
                    'sub_section': {}}
    if len(section.sections) > 1:
        sub_sections = section.sections[2:]
        for sub_sect in sub_sections:
            if sub_sect.title:
                if sub_sect.title not in title_ls:
                    title_ls.append(sub_sect.title)
                section_dict['sub_section'][sub_sect.title] = sub_sect.contents
                # section_dict['sub_section'][sub_sect.title] = parse_section(sub_sect, title_dict)
            else:
                section_dict['sub_section']['untitled'] = parse_section(sub_sect, title_dict)
    return section_dict

couple_writers = []
found_once = 0
count = 0
count_freq = {}
count_info = 0
title_dict = {}
with open('wikipedia/fr_dumps_wikipedia_writers.json') as json_file:
    data = json.load(json_file)
    for author in tqdm(author_ls, total=len(author_ls)):
        found = False
        count_freq_i = 0

        for title, text in data.items():
            normalised_title = normalize(title)
            dist_auteur = Levenshtein.distance(author, normalised_title)
            if dist_auteur < max(1, min(len(author), len(normalised_title)) / 6):
                # print('Auteur connus: ', author, ' avec ', title, '( normalisé: ', normalised_title, ')')
                count += 1
                count_freq_i += 1
                found = True
                new_couple = {
                        'author DB': author,
                        'author wiki': title,
                        'book DB': author_ls[author],
                    }

                info_box, page = split_info_box(text)

                new_couple['author wiki info_box'] = get_info(info_box)
                parsed_page = wtp.parse(page)
                sections_infos = {'untitled':[]}
                title_dict_couple = []
                for section in parsed_page.sections:
                    if section.title:
                        if section.title not in title_dict:
                            title_dict_couple.append(section.title)
                        test_parsing = parse_section(section, title_dict_couple)
                        sections_infos[section.title] = parse_section(section, title_dict_couple)
                    else:
                        sections_infos['untitled'].append(section.contents)
                new_couple['author wiki text parsing'] = sections_infos

                count_info += 1
                for headers in title_dict_couple:
                    if headers in title_dict:
                        title_dict[headers] += 1
                    else:
                        title_dict[headers] = 1


                couple_writers.append(new_couple)
        if found:
            found_once += 1
            count_freq[author] = count_freq_i
        if count_freq_i > 10:
            print("compte superieur a 10: ", author)

print("nombre d'autheurs en commun: ", found_once, " soit ", found_once * 100 / len(list(author_ls)), "%")
print("nombre d'autheurs reconnus: ", count, " soit ", count * 100 / len(list(author_ls)), "%")
print("Nombre d'information catégorisée: ", count_info, " soit ", count_info * 100 / len(list(author_ls)), "%")
print("Frequence de reconnaissance: ", count_freq)
print("titre des sections: ", sorted(title_dict.items(), key=lambda item: item[1], reverse=True))

with open('couple_wiki_authors.json', 'w') as outfile:
    json.dump(couple_writers, outfile)

100%|██████████| 21515/21515 [50:33<00:00,  7.09it/s] 


nombre d'autheurs en commun:  785  soit  3.648617243783407 %
nombre d'autheurs reconnus:  787  soit  3.657913083894957 %
Nombre d'information catégorisée:  787  soit  3.657913083894957 %
Frequence de reconnaissance:  {'Denis Monette': 1, 'Daniel Leblanc-Poirier': 1, 'Sylvain Hotte': 1, 'Michel Dufour': 1, 'Danielle Dussault': 1, 'Louise Simard': 1, 'Priska Poirier': 1, 'Fernand Dumont': 1, 'Chantal Bergeron': 1, 'Mario Bergeron': 1, 'Renaud Longchamps': 1, 'Chantal Gevrey': 1, 'Laurent Chabin': 1, 'Michel van Schendel': 1, 'Pierre Turgeon': 1, 'Mathieu Fortin': 1, 'Serge Gauthier': 1, 'Philippe Girard': 1, 'Gilles Marcotte': 1, 'Minette Walters': 1, 'Maxime Roussy': 1, 'Yolande Villemaire': 1, 'Marie Desjardins': 1, 'Jean-Pierre Gaudreau': 1, 'aude': 1, 'Madeleine Ouellette-Michalska': 1, 'Michèle Laframboise': 1, 'Gilbert Choquette': 1, 'Élisabeth Vonarburg': 1, 'Carole Tremblay': 1, 'Maude Veilleux': 1, 'Gabrielle Roy': 1, 'Reynald Cantin': 1, 'Gatien Lapointe': 1, 'Paulina Dachkova'

In [ ]:
# with open('couple_wiki_authors.json', 'r') as couples_file:
#     couple_writers = json.load(couples_file)

print('recherche des auteurs connus')
count = 0
for couple in couple_writers:
    for famous_author in famous:
        test = re.search(r"{}".format(famous_author), couple['author wiki'])
        if test:
            count += 1
            result = test.group()
            print('Auteur connus: ', couple["author wiki"])
            print(json.dumps(couple["author wiki data"], indent = 2))
print(count, ' auteurs trouvés, soit ', count * 100 / len(famous), '%')








